In [6]:
import requests
import urllib.request
import time
import os
import warnings
import re

from bs4 import BeautifulSoup

import csv
import os.path

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
def generateFileCSV(listHasil,csvName1):

	csvFolrder = "./"
	csvName = csvName1

	if os.path.exists(csvName):

		f = open(csvName, 'a', newline='\n')
		print(f)
		print("ada")
		w = csv.writer(f)

	else:

		f = open(csvName, 'w', newline='\n')
		print(f)
		print("tidak ada")

		w = csv.writer(f)
		w.writerow(("terdakwa", "penuntut_umum", "nomor", "tingkat_proses", "klasifikasi", "kata_kunci", "tahun", "tanggal_register",
              "lembaga_peradilan", "jenis_lembaga_peradilan", "hakim_ketua", "hakim_anggota", "panitera", "amar",
              "amar_lainnya", "catatan_amar", "tanggal_musyawarah", "tanggal_dibacakan", "kaidah", "abstrak", "url"))

	# menulis file csv
	for s in listHasil:
		w.writerow(s)

	f.close()
	berhasil = "\nCreate Csv file Berhasil\n"
	return berhasil

In [8]:
def generateMeta(urlMeta):

    url = str(urlMeta).strip()
    response = requests.get(url, verify=False)
    print(response)

    soup = BeautifulSoup(response.text, 'html.parser')
    cleanTags = re.compile('<.*?>')

    listMetaHead = ["terdakwa", "penuntut_umum", "nomor", "tingkat_proses", "klasifikasi", "kata_kunci", "tahun", "tanggal_register",
              "lembaga_peradilan", "jenis_lembaga_peradilan", "hakim_ketua", "hakim_anggota", "panitera", "amar",
              "amar_lainnya", "catatan_amar", "tanggal_musyawarah", "tanggal_dibacakan", "kaidah", "abstrak", "url"]

    # initialisasi
    listMeta = []

    rowsMETA1 = soup.find("ul", {"class": "portfolio-meta nobottommargin"}).find("table").findAll("tr")
    rowsMETA2 = soup.findAll("ul", {"class": "portfolio-meta nobottommargin"})
    for row in rowsMETA1:
        coll = row.findAll("td")

        cleantext2 =''
        cleantext1 =''

        if len(coll) > 1:
            cleantext2 = (re.sub(cleanTags, '', str(coll[1]))).strip()
            listMeta.append(cleantext2.replace('\n',' '))

        else:
            cleantext1 = (re.sub(cleanTags, ' ', str(coll[0]))).strip()
            # untuk putusan pidana akan muncul terdakwa dan penuntut umum pada meta
            # sedangkan untuk putusan selain pidana tidak muncul

            pidorpdt = re.search( r'(.*)/Pdt.(.*)',str(cleantext1), re.M|re.I)

            # check dokumen putusannya pidana atau perdata
            if pidorpdt == None:
                entTerdakwah = re.search( r'(.*)Terdakwa:(.*)',str(cleantext1), re.M|re.I)
                entPenuntut = re.search( r'Penuntut Umum:(.*)Terdakwa:',str(cleantext1), re.M|re.I)
            else:
                entTerdakwah = re.search( r'(.*)Tergugat:(.*)',str(cleantext1), re.M|re.I)
                entPenuntut = re.search( r'Penggugat:(.*)Tergugat:',str(cleantext1), re.M|re.I)

            if entTerdakwah == None:
                listMeta.append("")
            else:
                listMeta.append(entTerdakwah.group(2))

            if entPenuntut == None:
                listMeta.append("")
            else:
                listMeta.append(entPenuntut.group(1))

    urlDL = rowsMETA2[1].findAll("li")
    urlDLStr = str(urlDL[4])
    listMeta.append(urlDLStr[urlDLStr.find("https"):urlDLStr.find('">')])

    return listMeta

In [9]:
def main():

    warnings.filterwarnings('ignore')

    # fileListURL = "/content/drive/MyDrive/KULIAH/SEMESTER 5/EKSTRAKSI INFORMASI/TUGAS IE 01 - SCRAPPING/Situs_PN Pekalongan_Harits.txt"
    fileListURL = "/content/drive/MyDrive/KULIAH/SEMESTER 5/EKSTRAKSI INFORMASI/TUGAS IE 01 - SCRAPPING/text.txt"
    fileMetaCSV = "/content/drive/MyDrive/KULIAH/SEMESTER 5/EKSTRAKSI INFORMASI/TUGAS IE 01 - SCRAPPING/metaPidanaUmumPNPekalongan.csv"
    listHasil =[]


    startTime = time.time()
    openfileListURL = open(fileListURL, "r", encoding='UTF8')
    bacaListURL = openfileListURL.readlines()

    i = 1
    for barisURL in bacaListURL:
        try:
            #print(str(barisURL))
            hasil = generateMeta(str(barisURL))
            listHasil.append(hasil)
            print("======= ROW HASIL =======",i)
            #print(hasil)
        except Exception as e:
            print(f"Error Get Meta Inf, {e}")
        i=i+1

    createFile = generateFileCSV(listHasil,fileMetaCSV)

    openfileListURL.close()
    endTime = time.time()
    #print(listHasil)
    print('Time Processing : ', endTime-startTime, ' Second')

main();



<Response [200]>
======= ROW HASIL ======= 1
<Response [200]>
======= ROW HASIL ======= 2
<_io.TextIOWrapper name='/content/drive/MyDrive/KULIAH/SEMESTER 5/EKSTRAKSI INFORMASI/TUGAS IE 01 - SCRAPPING/metaPidanaUmumPNPekalongan.csv' mode='w' encoding='utf-8'>
tidak ada
Time Processing :  47.785868883132935  Second
